In [102]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd 
import pandas_ta as ta # for data analysis and calculation of technical indcator
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
import time
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [103]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'
mt5.login(login,password,server)

True

In [104]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df

In [ ]:
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_H4
start_datetime = datetime(2024,12,15)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# hour4 df
timeframe2 = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe2, start_datetime, end_datetime)
# daily df
timeframe3 = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe3, start_datetime, end_datetime)

In [ ]:
def mystrategy():
    symbol = 'EURUSD'
    pos_size = 1
    timeframe = mt5.TIMEFRAME_H4
    start_datetime = datetime(2024,12,15)
    end_datetime = datetime.now()
    ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
    # hour4 df
    timeframe2 = mt5.TIMEFRAME_H4
    hourly_df = get_ohlc(symbol, timeframe2, start_datetime, end_datetime)
    # daily df
    timeframe3 = mt5.TIMEFRAME_D1
    daily_df = get_ohlc(symbol, timeframe3, start_datetime, end_datetime)
    #ohlc4
    daily_df['ohlc'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
    hourly_df['ohlc'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4
    #sma
    daily_df['sma_1'] = ta.sma(close = daily_df['ohlc'], length= 1 )
    daily_df['sma_14'] = ta.sma(close = daily_df['ohlc'], length= 14)
    hourly_df['sma_11'] = ta.sma(close = hourly_df['ohlc'], length= 1)
    hourly_df['sma_144'] = ta.sma(close = hourly_df['ohlc'], length= 14)
    daily_df['high_20'] = ta.sma(close = daily_df['high'], length= 20 )
    daily_df['low_20'] = ta.sma(close = daily_df['low'], length= 20)
    def entring_stoploss_profit(row, hourly_df) :
        hour = row['time'].hour
        if row['sma_1'] > row['sma_14'] and row["sma_11"] > row['sma_144'] :#and(hour > 8):
            return "buy"
        elif row['sma_1'] < row['sma_14'] and row['sma_11'] < row['sma_144'] :
            return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
    merged_df1 = pd.merge_asof(ohlc_df, hourly_df, on='time', direction='backward')
    merged_df = pd.merge_asof(merged_df1, daily_df, on='time', direction='backward')
    merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, hourly_df = hourly_df)

    # Tag first signal per day in original merged_df
    merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

    # Set signal only on the first row per day, others = None
    merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
    merged_df[['signal', 'time']].dropna()

,signal,time
78,sell,2025-01-06 00:00:00
89,sell,2025-01-07 20:00:00
90,sell,2025-01-08 00:00:00
96,sell,2025-01-09 00:00:00
102,sell,2025-01-10 00:00:00
...,...,...
1206,sell,2025-09-25 00:00:00
1212,sell,2025-09-26 00:00:00
1234,sell,2025-10-01 16:00:00
1236,sell,2025-10-02 00:00:00


In [108]:
import schedule
#schedule.every(120).seconds.do(excution)

print("------> Strategy scheduler is running every 60 seconds...")

"""while True:
    schedule.run_pending()
    time.sleep(60)"""

------> Strategy scheduler is running every 60 seconds...


'while True:\n    schedule.run_pending()\n    time.sleep(60)'

In [109]:
merged_df = merged_df.rename(columns={
    "low_x": "low",
    "high_x": "high",
    "open_x": "open",
    "close_x": "close",
    "tick_volume_x": "tick_volume",
    "spread_x" : "spread",
    "real_volume_x" : "real_volume",
    "low_y": "hourly_low",
    "high_y": "hourly_high",
    "open_y": "hourly_open",
    "close_y": "hourly_close",
    "low": "daily_low",
    "high": "daily_high",
    "open": "daily_open",
    "close": "daily_close",
    "tick_volume": "daily_tick_volume",
    "spread" : "daily_spread",
    "real_volume" : "daily_real_volume"
})
print(merged_df.columns.tolist())
merged_df


['time', 'open', 'high', 'low', 'close', 'tick_volume', 'spread', 'real_volume', 'hourly_open', 'hourly_high', 'hourly_low', 'hourly_close', 'tick_volume_y', 'spread_y', 'real_volume_y', 'ohlc_x', 'sma_11', 'sma_144', 'daily_open', 'daily_high', 'daily_low', 'daily_close', 'daily_tick_volume', 'daily_spread', 'daily_real_volume', 'ohlc_y', 'sma_1', 'sma_14', 'high_20', 'low_20', 'signal', 'is_first_signal']


,time,open,high,low,close,tick_volume,spread,real_volume,hourly_open,hourly_high,...,daily_tick_volume,daily_spread,daily_real_volume,ohlc_y,sma_1,sma_14,high_20,low_20,signal,is_first_signal
0,2024-12-16 00:00:00,1.04850,1.05213,1.04797,1.05190,6402,0,0,1.04850,1.05213,...,53594,0,0,1.049895,1.049895,NaN,NaN,NaN,None,NaN
1,2024-12-16 04:00:00,1.05191,1.05230,1.05098,1.05196,4883,0,0,1.05191,1.05230,...,53594,0,0,1.049895,1.049895,NaN,NaN,NaN,None,NaN
2,2024-12-16 08:00:00,1.05196,1.05248,1.05003,1.05051,11238,0,0,1.05196,1.05248,...,53594,0,0,1.049895,1.049895,NaN,NaN,NaN,None,NaN
3,2024-12-16 12:00:00,1.05051,1.05084,1.04833,1.05078,9652,0,0,1.05051,1.05084,...,53594,0,0,1.049895,1.049895,NaN,NaN,NaN,None,NaN
4,2024-12-16 16:00:00,1.05075,1.05207,1.04746,1.05112,15401,0,0,1.05075,1.05207,...,53594,0,0,1.049895,1.049895,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,2025-10-03 00:00:00,1.17116,1.17260,1.17075,1.17243,3834,0,0,1.17116,1.17260,...,47756,0,0,1.173005,1.173005,1.175694,1.17849,1.171285,sell,True
1243,2025-10-03 04:00:00,1.17244,1.17293,1.17163,1.17181,6061,0,0,1.17244,1.17293,...,47756,0,0,1.173005,1.173005,1.175694,1.17849,1.171285,None,False
1244,2025-10-03 08:00:00,1.17181,1.17446,1.17149,1.17402,9122,0,0,1.17181,1.17446,...,47756,0,0,1.173005,1.173005,1.175694,1.17849,1.171285,None,False
1245,2025-10-03 12:00:00,1.17401,1.17429,1.17274,1.17395,9111,0,0,1.17401,1.17429,...,47756,0,0,1.173005,1.173005,1.175694,1.17849,1.171285,None,NaN


In [110]:
# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        #stop_loss_pips = (data['high'] - data["low_20"]) # example SL
        volume = 50000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        #stop_loss_pips = (data['high_20'] - data["low_20"]) # example SL
        volume = 50000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["sma_11"] < data["sma_144"] :
            orders.close_trade(trade) 
        elif trade['order_type'] == 'sell' and data["sma_11"] > data["sma_144"] :
            orders.close_trade(trade)


In [111]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 50000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(merged_df)
bt.set_on_bar(on_bar)

get_ohlc_history(symbol, timeframe, start_datetime, end_datetime)
bt.run_backtest()


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,sell,50000,2025-01-06 00:00:00,1.03021,2025-01-06 04:00:00,1.03123,0,0,{},-51.0,-7.0,-58.0,-58.0,9942.0
1,closed,EURUSD,sell,50000,2025-01-07 20:00:00,1.03671,2025-01-14 00:00:00,1.02443,0,0,{},614.0,-7.0,607.0,549.0,10549.0
2,closed,EURUSD,sell,50000,2025-01-14 04:00:00,1.02492,2025-01-14 08:00:00,1.02449,0,0,{},21.5,-7.0,14.5,563.5,10563.5
3,closed,EURUSD,sell,50000,2025-01-16 12:00:00,1.02862,2025-01-16 20:00:00,1.02966,0,0,{},-52.0,-7.0,-59.0,504.5,10504.5
4,closed,EURUSD,sell,50000,2025-01-17 08:00:00,1.02933,2025-01-20 04:00:00,1.02909,0,0,{},12.0,-7.0,5.0,509.5,10509.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,closed,EURUSD,buy,50000,2025-09-11 16:00:00,1.17157,2025-09-18 00:00:00,1.18123,0,0,{},483.0,-7.0,476.0,13815.5,23815.5
65,closed,EURUSD,buy,50000,2025-09-22 12:00:00,1.17624,2025-09-24 08:00:00,1.18029,0,0,{},202.5,-7.0,195.5,14011.0,24011.0
66,closed,EURUSD,sell,50000,2025-09-25 00:00:00,1.17374,2025-09-29 00:00:00,1.16984,0,0,{},195.0,-7.0,188.0,14199.0,24199.0
67,closed,EURUSD,sell,50000,2025-10-01 16:00:00,1.1749,2025-10-02 08:00:00,1.17344,0,0,{},73.0,-7.0,66.0,14265.0,24265.0


In [116]:
def excution():
    tick = mt5.symbol_info_tick(symbol)
    lot = 10000
    price = tick.ask
    buy_order_type = mt5.ORDER_TYPE_BUY
    sell_order_type = mt5.ORDER_TYPE_SELL
    if merged_df["signal"].iloc[-1] not in ["buy", "sell"]:
        print("No trade signal (neutral) — skipping trade.")
        return
    if hourly_df["sma_11"].iloc[-1] < hourly_df["sma_144"].iloc[-1]:
            buy_sl = ohlc_df["close"].iloc[-1]
    if hourly_df["sma_11"].iloc[-1] > hourly_df["sma_144"].iloc[-1]:
            sell_sl = ohlc_df["close"].iloc[-1]
    if merged_df["signal"].iloc[-1] == "buy" and not buy_order_type.shape[0] :
        request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": buy_order_type,
        "price": price,
        "sl": buy_sl,
        #"tp": tp,
        "deviation": 10,
        "magic": 234000,
        "comment": f"buy executed {price}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
        result = mt5.order_send(request)

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print(f"buy trade placed successfully at {price}")
            print(f"SL: {buy_sl}")
        
    elif merged_df["signal"].iloc[-1] == "sell" and not sell_order_type.shape[0] :
        request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": sell_order_type,
        "price": price,
        "sl": sell_sl,
        #"tp": tp,
        "deviation": 10,
        "magic": 234000,
        "comment": f"sell executed {price}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
        result = mt5.order_send(request)

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print(f"sell trade placed successfully at {price}")
            print(f"SL: {sell_sl}")
excution()

No trade signal (neutral) — skipping trade.


In [113]:
mm = bt.plot_balance()
display(mm)

In [114]:
evaluate_backtest(bt.trades)

Daily Drawdown : -605.00
Max Drawdown (portfolio): -605.00
worst trades (trade MAE): [np.float64(-80.99999999999774), np.float64(-12.000000000000899), np.float64(-128.49999999999807), np.float64(-144.99999999999514), np.float64(-186.99999999999272), np.float64(-368.00000000000165), np.float64(0.0), np.float64(-107.49999999999372), np.float64(-229.00000000000142), np.float64(-136.99999999999824), np.float64(-110.99999999999443), np.float64(-131.4999999999955), np.float64(-45.49999999999832), np.float64(-30.49999999999997), np.float64(-39.500000000003425), np.float64(-393.9999999999944), np.float64(-73.99999999999629), np.float64(-34.99999999999615), np.float64(-35.00000000000725), np.float64(-59.50000000000122), np.float64(-332.5000000000022), np.float64(-89.00000000000574), np.float64(-276.50000000000176), np.float64(-229.00000000000142), np.float64(-197.4999999999949), np.float64(-100.99999999999554), np.float64(-54.50000000000177), np.float64(-165.9999999999995), np.float64(-114.9999

,order_type,profit
0,buy,11148.0
1,sell,3450.5


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown
1,closed,EURUSD,sell,50000,2025-01-07 20:00:00,1.03671,2025-01-14 00:00:00,1.02443,0,0,...,614.0,-7.0,607.0,549.0,10549.0,2025-01-07,2025-01-14 00:00:00,-12.0,549.0,0.0
2,closed,EURUSD,sell,50000,2025-01-14 04:00:00,1.02492,2025-01-14 08:00:00,1.02449,0,0,...,21.5,-7.0,14.5,563.5,10563.5,2025-01-14,2025-01-14 08:00:00,-128.5,563.5,0.0
4,closed,EURUSD,sell,50000,2025-01-17 08:00:00,1.02933,2025-01-20 04:00:00,1.02909,0,0,...,12.0,-7.0,5.0,509.5,10509.5,2025-01-17,2025-01-20 04:00:00,-187.0,563.5,-54.0
6,closed,EURUSD,buy,50000,2025-01-24 04:00:00,1.04159,2025-01-28 00:00:00,1.04915,0,0,...,378.0,-7.0,371.0,813.5,10813.5,2025-01-24,2025-01-28 00:00:00,0.0,813.5,0.0
7,closed,EURUSD,buy,50000,2025-01-30 12:00:00,1.04077,2025-01-30 20:00:00,1.04173,0,0,...,48.0,-7.0,41.0,854.5,10854.5,2025-01-30,2025-01-30 20:00:00,-107.5,854.5,0.0
8,closed,EURUSD,sell,50000,2025-01-31 00:00:00,1.03882,2025-02-04 00:00:00,1.03436,0,0,...,223.0,-7.0,216.0,1070.5,11070.5,2025-01-31,2025-02-04 00:00:00,-229.0,1070.5,0.0
9,closed,EURUSD,sell,50000,2025-02-04 04:00:00,1.03221,2025-02-04 08:00:00,1.02932,0,0,...,144.5,-7.0,137.5,1208.0,11208.0,2025-02-04,2025-02-04 08:00:00,-137.0,1208.0,0.0
10,closed,EURUSD,buy,50000,2025-02-05 00:00:00,1.03785,2025-02-06 08:00:00,1.03864,0,0,...,39.5,-7.0,32.5,1240.5,11240.5,2025-02-05,2025-02-06 08:00:00,-111.0,1240.5,0.0
11,closed,EURUSD,sell,50000,2025-02-07 00:00:00,1.03831,2025-02-11 16:00:00,1.03225,0,0,...,303.0,-7.0,296.0,1536.5,11536.5,2025-02-07,2025-02-11 16:00:00,-131.5,1536.5,0.0
12,closed,EURUSD,buy,50000,2025-02-13 00:00:00,1.03823,2025-02-18 00:00:00,1.04836,0,0,...,506.5,-7.0,499.5,2036.0,12036.0,2025-02-13,2025-02-18 00:00:00,-45.5,2036.0,0.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown
0,closed,EURUSD,sell,50000,2025-01-06 00:00:00,1.03021,2025-01-06 04:00:00,1.03123,0,0,...,-51.0,-7.0,-58.0,-58.0,9942.0,2025-01-06,2025-01-06 04:00:00,-81.0,-58.0,0.0
3,closed,EURUSD,sell,50000,2025-01-16 12:00:00,1.02862,2025-01-16 20:00:00,1.02966,0,0,...,-52.0,-7.0,-59.0,504.5,10504.5,2025-01-16,2025-01-16 20:00:00,-145.0,563.5,-59.0
5,closed,EURUSD,buy,50000,2025-01-21 00:00:00,1.04156,2025-01-23 04:00:00,1.04036,0,0,...,-60.0,-7.0,-67.0,442.5,10442.5,2025-01-21,2025-01-23 04:00:00,-368.0,563.5,-121.0
15,closed,EURUSD,sell,50000,2025-02-28 00:00:00,1.03976,2025-03-03 12:00:00,1.04275,0,0,...,-149.5,-7.0,-156.5,2096.5,12096.5,2025-02-28,2025-03-03 12:00:00,-394.0,2253.0,-156.5
20,closed,EURUSD,sell,50000,2025-03-28 12:00:00,1.07783,2025-03-28 16:00:00,1.07891,0,0,...,-54.0,-7.0,-61.0,4693.5,14693.5,2025-03-28,2025-03-28 16:00:00,-332.5,4754.5,-61.0
21,closed,EURUSD,sell,50000,2025-03-31 16:00:00,1.08036,2025-03-31 20:00:00,1.08118,0,0,...,-41.0,-7.0,-48.0,4645.5,14645.5,2025-03-31,2025-03-31 20:00:00,-89.0,4754.5,-109.0
22,closed,EURUSD,sell,50000,2025-04-01 12:00:00,1.08177,2025-04-02 16:00:00,1.08192,0,0,...,-7.5,-7.0,-14.5,4631.0,14631.0,2025-04-01,2025-04-02 16:00:00,-276.5,4754.5,-123.5
24,closed,EURUSD,buy,50000,2025-04-09 00:00:00,1.09527,2025-04-10 00:00:00,1.09483,0,0,...,-22.0,-7.0,-29.0,5197.0,15197.0,2025-04-09,2025-04-10 00:00:00,-197.5,5226.0,-29.0
29,closed,EURUSD,buy,50000,2025-04-29 00:00:00,1.14197,2025-04-30 04:00:00,1.13827,0,0,...,-185.0,-7.0,-192.0,7452.0,17452.0,2025-04-29,2025-04-30 04:00:00,-323.5,7644.0,-192.0
30,closed,EURUSD,sell,50000,2025-05-01 00:00:00,1.13279,2025-05-02 12:00:00,1.1333,0,0,...,-25.5,-7.0,-32.5,7419.5,17419.5,2025-05-01,2025-05-02 12:00:00,-136.0,7644.0,-224.5


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,balance,date,time,intrabar_drawdown,current_max,drawdown,dayofweek,hourofday,month,year
0,closed,EURUSD,sell,50000,2025-01-06 00:00:00,1.03021,2025-01-06 04:00:00,1.03123,0,0,...,9942.0,2025-01-06,2025-01-06 04:00:00,-81.0,-58.0,0.0,0,0,2025-01,2025
1,closed,EURUSD,sell,50000,2025-01-07 20:00:00,1.03671,2025-01-14 00:00:00,1.02443,0,0,...,10549.0,2025-01-07,2025-01-14 00:00:00,-12.0,549.0,0.0,1,20,2025-01,2025
2,closed,EURUSD,sell,50000,2025-01-14 04:00:00,1.02492,2025-01-14 08:00:00,1.02449,0,0,...,10563.5,2025-01-14,2025-01-14 08:00:00,-128.5,563.5,0.0,1,4,2025-01,2025
3,closed,EURUSD,sell,50000,2025-01-16 12:00:00,1.02862,2025-01-16 20:00:00,1.02966,0,0,...,10504.5,2025-01-16,2025-01-16 20:00:00,-145.0,563.5,-59.0,3,12,2025-01,2025
4,closed,EURUSD,sell,50000,2025-01-17 08:00:00,1.02933,2025-01-20 04:00:00,1.02909,0,0,...,10509.5,2025-01-17,2025-01-20 04:00:00,-187.0,563.5,-54.0,4,8,2025-01,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,closed,EURUSD,buy,50000,2025-09-11 16:00:00,1.17157,2025-09-18 00:00:00,1.18123,0,0,...,23815.5,2025-09-11,2025-09-18 00:00:00,-72.5,13815.5,0.0,3,16,2025-09,2025
65,closed,EURUSD,buy,50000,2025-09-22 12:00:00,1.17624,2025-09-24 08:00:00,1.18029,0,0,...,24011.0,2025-09-22,2025-09-24 08:00:00,-4.0,14011.0,0.0,0,12,2025-09,2025
66,closed,EURUSD,sell,50000,2025-09-25 00:00:00,1.17374,2025-09-29 00:00:00,1.16984,0,0,...,24199.0,2025-09-25,2025-09-29 00:00:00,-83.0,14199.0,0.0,3,0,2025-09,2025
67,closed,EURUSD,sell,50000,2025-10-01 16:00:00,1.1749,2025-10-02 08:00:00,1.17344,0,0,...,24265.0,2025-10-01,2025-10-02 08:00:00,-72.5,14265.0,0.0,2,16,2025-10,2025


max_drawdown -386.0
